In [ ]:
#default_exp data

# Data

> This page takes through the steps of the queries used, data collected and saved. The data saved in these steps is used for the analysis and modeling. 

The data is saved in the following format:
- `nbs/data/raw` - The original, immutable data dump.
- `nbs/data/interim` folder - Intermediate data that has been transformed.
- `raw/data/processed` folder - The final, canonical data sets for modeling.
- `raw/data/external` folder - Data from external sources (outside snowflake or in excel sheets or csv files)

In [ ]:

#hide
import os
from nbdev.showdoc import *
from dotenv import load_dotenv

development = False

# load the user and password for snowflake"
if development:
    load_dotenv("../.secrets")
    password = os.environ["MY_SNOWFLAKE_PASSWORD"]
    user = os.environ["MY_SNOWFLAKE_USER"]

In [ ]:
#hide
import os
import snowflake.connector

# Data related
import pandas as pd
import numpy as np

# multiple outputs for cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotting
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# plotting options
plt.style.use('default')

# plotting output options
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext blackcellmagic

use_snowflake = False # set to False if the data is available the nbs/data subfolders


## Connection

In [ ]:
#hide
if use_snowflake:

    con = snowflake.connector.connect(
    account="shipt.us-east-1",
    user=user,
    password=password,
    database="PRD_DATALAKEHOUSE",
    warehouse="PRD_DATA_SCIENTIST_WHS",
    authenticator="https://shipt.okta.com"
    )

## Filters

December 2021 data for Detroit metro and Meijer store is used for the analysis

In [ ]:
Month = '2021-12-01'
Metro = 'Detroit'
Retailer = 'Meijer'

## Order financials 

### Query

In [ ]:
order_financials_query = """
SELECT date_trunc(
    'month', convert_timezone('US/Central', delivered_at_utc)
    )::date AS MONTH,
    M.NAME AS METRO,
    s.name AS RETAILER,
    SUM(GMV) AS GMV,
    COUNT(DISTINCT o.id) AS ORDER_VOLUME
FROM PRD_DATALAKEHOUSE.DATA_SCIENCE.ORDER_FINANCIALS o
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.METROS m ON m.id = o.metro_id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.STORES s ON s.id = o.store_id
WHERE o.status ='delivered'
  AND convert_timezone(
    'UTC', 'US/Central', DELIVERED_AT_UTC::timestamp_ntz
    )::date BETWEEN '2020-01-01' AND '2021-12-31'
  AND o.order_type = 'Marketplace'
GROUP BY MONTH,
         METRO,
         RETAILER;
"""

### Save data

In [ ]:
ls

00_data.ipynb                data/
01_analysis.ipynb            index.ipynb
02_models.ipynb              models/
03_feature_importance.ipynb  reports/
04_validation.ipynb


In [ ]:
#hide
if use_snowflake:
    df = pd.read_sql(con=con, sql = order_financials_query)
    df['MONTH'] = pd.to_datetime(df['MONTH'])
    df.to_parquet('data/raw/order_financials.parquet')
else:
    df = pd.read_parquet('data/raw/order_financials.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/order_financials.parquet'

In [ ]:
df.shape
df.head()

(45357, 5)

,MONTH,METRO,RETAILER,GMV,ORDER_VOLUME
0,2021-12-01,Detroit,Target,831536.49,10846
1,2021-12-01,Fort Myers / Cape Coral,Publix,590075.68,5067
2,2021-01-01,Austin,Target,311242.01,4124
3,2021-12-01,Reading,Target,54143.47,682
4,2021-06-01,The Triad,Costco,40810.46,294


## Average metrics

### Query

In [ ]:
avg_metrics_query = """
SELECT PRODUCT_ID,
       PRODUCT_NAME,
       AVG(GMV) AS AVG_GMV, 
       AVG(ORDER_LINE_COUNT) AS AVG_ORDERLINES,
       AVG(QTY_SOLD) AS AVG_QTY,
       AVG(COST_SUM) AS AVG_COST,
       AVG(PRICE_SUM) AS AVG_PRICE,
       AVG(ORD_CUST_COUNT) AS AVG_CUST_COUNT
FROM
  (SELECT convert_timezone(
    'UTC', 'US/Central', o.DELIVERED_AT::timestamp_ntz
    )::date AS DEL_DATE,
          M.NAME AS METRO,
          s.name AS RETAILER,
          ol.actual_product_id AS PRODUCT_ID,
          p.NAME AS PRODUCT_NAME, 
          SUM(saved_product_price * actual_qty) AS GMV,
          COUNT(DISTINCT o.id) AS ORDER_VOLUME,
          COUNT(DISTINCT ol.id) AS ORDER_LINE_COUNT,
          SUM(ol.actual_qty) AS QTY_SOLD,
          SUM(ol.saved_product_cost) AS COST_SUM,
          SUM(ol.saved_product_price) AS PRICE_SUM,
          COUNT(DISTINCT o.CUSTOMER_ID) AS ORD_CUST_COUNT
   FROM PRD_DATALAKEHOUSE.OG_VIEWS.ORDER_LINES ol
   INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.ORDERS o ON ol.order_id = o.id
   INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.METROS m ON m.id = o.metro_id
   INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.STORES s ON s.id = o.store_id
   INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.PRODUCTS p ON p.id = ol.actual_product_id
   WHERE o.status ='delivered'
     AND ACTUAL_PRODUCT_TYPE = 'Product'
     AND is_external_platform_order = FALSE
     AND partner_id IS NULL
     AND convert_timezone(
       'UTC', 'US/Central', DELIVERED_AT::timestamp_ntz
       )::date BETWEEN '2021-12-01' AND '2021-12-31'
     AND m.name = 'Detroit'
     AND s.name = 'Meijer'
   GROUP BY DEL_DATE,
            METRO,
            RETAILER,
            PRODUCT_ID,
            PRODUCT_NAME)
GROUP BY PRODUCT_ID,
         PRODUCT_NAME
ORDER BY PRODUCT_NAME;
"""

### Save data

In [ ]:
#hide
if use_snowflake:
    df_avg_metrics = pd.read_sql(con=con, sql = avg_metrics_query)
    df_avg_metrics.to_parquet('data/raw/avg_metrics.parquet')
else:
    df_avg_metrics = pd.read_parquet('data/raw/avg_metrics.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
df_avg_metrics.shape
df_avg_metrics.head()

(49694, 8)

,PRODUCT_ID,PRODUCT_NAME,AVG_GMV,AVG_ORDERLINES,AVG_QTY,AVG_COST,AVG_PRICE,AVG_CUST_COUNT
0,10286076,"""\tNOBLETREE Reverence Espresso Whole Bean Cof...",10.49,1.0,1.0,8.99,10.49,1.0
1,10433693,""" Handcrafted Ornament Set, 1"""" RoundDecember ...",7.59,1.0,1.0,6.99,7.59,1.0
2,10438190,""" Handcrafted Ornament Set, 1"""" RoundDecember ...",7.59,1.0,1.0,6.99,7.59,1.0
3,10511929,""" Handcrafted Ornament Set, 1"""" RoundDecember ...",12.18,1.0,2.0,5.59,6.09,1.0
4,10548473,""" Handcrafted Ornament Set, 1.5"""" RoundDecembe...",12.99,1.0,1.0,11.99,12.99,1.0


## Click stream metrics

Metrics grouped by product_id, product_name

Q: The metrics are at Product Id and Product name level for the entire month ??
- `avg_cart_add_daily` and `ADD_TO_ORDER_daily` are not actually daily measures. These metrics are for the entire month for each `product_id` and `product_name`

### Query

In [ ]:
clickstream_metrics_query = """
SELECT DISTINCT product_id,
                product_name,
                AVG(ATC_count) AS avg_cart_add_daily,
                AVG(ADD_TO_ORDER_COUNT) AS ADD_TO_ORDER_daily
FROM
  (SELECT DISTINCT event_stamp::date AS event_date,
          product_id,
          product_name, 
          COUNT(DISTINCT(CASE 
            WHEN event_type = 'product_added_to_cart' 
            THEN EVENT_ID END)) 
          AS ATC_count,
          COUNT(DISTINCT(CASE 
            WHEN event_type = 'product_added_to_order' 
            THEN EVENT_ID END)) 
          AS ADD_TO_ORDER_COUNT
   FROM PRD_SEGMENT_EVENTS.MEMBER.SEARCH_BROWSE_CONVERSIONS
   WHERE metro = 'Detroit'
     AND store = 'Meijer'
     AND event_stamp::date BETWEEN '2021-12-01' AND '2021-12-31'
   GROUP BY event_date,
            product_id,
            product_name
   ORDER BY ATC_COUNT DESC)
GROUP BY product_id,
         product_name;
"""

### Save data

In [ ]:
#hide
if use_snowflake:
    df_clickstream_metrics = pd.read_sql(con=con, sql = clickstream_metrics_query)
    df_clickstream_metrics.to_parquet('data/raw/clickstream_metrics.parquet')
else:
    df_clickstream_metrics = pd.read_parquet('data/raw/clickstream_metrics.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
df_clickstream_metrics.shape
df_clickstream_metrics.head()

(56916, 4)

,PRODUCT_ID,PRODUCT_NAME,AVG_CART_ADD_DAILY,ADD_TO_ORDER_DAILY
0,4251091,Hudsonville Ice Cream Triple Peanut Butter Cup,2.608696,0.086957
1,3967404,Hebrew National Bun Length Beef Franks,3.500000,0.035714
2,3972759,Thai Kitchen Gluten Free Unsweetened Coconut C...,1.647059,0.176471
3,4039320,bubly Grapefruit Sparkling Water,2.263158,0.105263
4,3985119,Pillsbury Chocolate Chip Cookie Dough,3.250000,0.107143


## Growth metrics

Metrics grouped by `DATE_MONTH`, product_id and product_name

### Query

In [ ]:
growth_metrics_query = """
SELECT month(
  convert_timezone(
    'UTC', 'US/Central', DELIVERED_AT::timestamp_ntz)::date
    ) AS date_month,
    ol.actual_product_id AS PRODUCT_ID,
    p.NAME AS PRODUCT_NAME, SUM(saved_product_price * actual_qty) AS GMV, 
    COUNT(DISTINCT ol.id) AS ORDER_LINE_COUNT,
    SUM(ol.actual_qty) AS QTY_SOLD,
    COUNT(DISTINCT o.CUSTOMER_ID) AS ORD_CUST_COUNT
FROM PRD_DATALAKEHOUSE.OG_VIEWS.ORDER_LINES ol
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.ORDERS o ON ol.order_id = o.id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.METROS m ON m.id = o.metro_id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.STORES s ON s.id = o.store_id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.PRODUCTS p ON p.id = ol.actual_product_id
WHERE o.status ='delivered'
  AND ACTUAL_PRODUCT_TYPE = 'Product'
  AND is_external_platform_order = FALSE
  AND partner_id IS NULL
  AND convert_timezone(
    'UTC', 'US/Central', DELIVERED_AT::timestamp_ntz
    )::date BETWEEN '2021-01-01' AND '2021-12-31'
  AND m.name = 'Detroit'
  AND s.name = 'Meijer'
GROUP BY DATE_MONTH,
         PRODUCT_ID,
         PRODUCT_NAME;
"""

###  Save data - raw

In [ ]:
#hide
if use_snowflake:
    df_growth_metrics = pd.read_sql(con=con, sql = growth_metrics_query)
    df_growth_metrics.to_parquet('data/raw/growth_metrics.parquet')
else:
    df_growth_metrics = pd.read_parquet('data/raw/growth_metrics.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
df_growth_metrics.shape
df_growth_metrics.head()

(596110, 7)

,DATE_MONTH,PRODUCT_ID,PRODUCT_NAME,GMV,ORDER_LINE_COUNT,QTY_SOLD,ORD_CUST_COUNT
0,9,4003519,Mashed Cauliflower,376.60,53,60.0,45
1,12,3974692,Q-Tips Cotton Swabs,1271.99,301,311.0,298
2,10,3982786,Dijon Mustard,214.21,145,149.0,142
3,12,3987097,Lemon Lime Soda Bottle,62.14,19,26.0,18
4,9,6219504,"Eggland's Best Grade A Extra Large Eggs, 1 Dozen",530.95,127,155.0,112


### Derive growth metrics

Bringing growth metrics at product level.

The following metrics are added
1. `GMV_growth`
2. `Order_line_growth`
3. `Qty_sold_growth`
4. `Ord_Cust_growth`

For each metric, the values are sorted based on `DATE_MONTH` and then grouped by `PRODUCT_ID` and respecive column (`GMV`, `ORDER_LINE_COUNT` etc.). Then, growth is computed by percentage change from the previous period


_Example_

```python
df_growth_metrics["GMV_growth"] = (
    df_growth_metrics[["DATE_MONTH", "PRODUCT_ID", "GMV"]]
    .sort_values(by="DATE_MONTH")
    .groupby(["PRODUCT_ID"])["GMV"]
    .pct_change(periods=1, fill_method="ffill")
)
```


In [ ]:
#hide

df_growth_metrics["GMV_growth"] = (
    df_growth_metrics[["DATE_MONTH", "PRODUCT_ID", "GMV"]]
    .sort_values(by="DATE_MONTH")
    .groupby(["PRODUCT_ID"])["GMV"]
    .pct_change(periods=1, fill_method="ffill")
)
df_growth_metrics["Order_line_growth"] = (
    df_growth_metrics[["DATE_MONTH", "PRODUCT_ID", "ORDER_LINE_COUNT"]]
    .sort_values(by="DATE_MONTH")
    .groupby(["PRODUCT_ID"])["ORDER_LINE_COUNT"]
    .pct_change(periods=1, fill_method="ffill")
)
df_growth_metrics["Qty_sold_growth"] = (
    df_growth_metrics[["DATE_MONTH", "PRODUCT_ID", "QTY_SOLD"]]
    .sort_values(by="DATE_MONTH")
    .groupby(["PRODUCT_ID"])["QTY_SOLD"]
    .pct_change(periods=1, fill_method="ffill")
)
df_growth_metrics["Ord_Cust_growth"] = (
    df_growth_metrics[["DATE_MONTH", "PRODUCT_ID", "ORD_CUST_COUNT"]]
    .sort_values(by="DATE_MONTH")
    .groupby(["PRODUCT_ID"])["ORD_CUST_COUNT"]
    .pct_change(periods=1, fill_method="ffill")
)

In [ ]:
df_growth_metrics.shape
df_growth_metrics.head().T
df_growth_metrics['DATE_MONTH'].value_counts()

(596110, 11)

,0,1,2,3,4
DATE_MONTH,9,12,10,12,9
PRODUCT_ID,4003519,3974692,3982786,3987097,6219504
PRODUCT_NAME,Mashed Cauliflower,Q-Tips Cotton Swabs,Dijon Mustard,Lemon Lime Soda Bottle,"Eggland's Best Grade A Extra Large Eggs, 1 Dozen"
GMV,376.6,1271.99,214.21,62.14,530.95
ORDER_LINE_COUNT,53,301,145,19,127
QTY_SOLD,60.0,311.0,149.0,26.0,155.0
ORD_CUST_COUNT,45,298,142,18,112
GMV_growth,0.415896,-0.022591,-0.236056,0.083333,0.305187
Order_line_growth,0.394737,0.003333,-0.08805,-0.136364,0.27
Qty_sold_growth,0.428571,0.0,-0.06875,0.083333,0.291667


1     51255
3     51128
4     50993
5     50901
6     49842
12    49694
2     49649
8     49388
7     49071
9     48204
11    48005
10    47980
Name: DATE_MONTH, dtype: int64

In [ ]:
#hide
df_growth_final = df_growth_metrics.groupby(['PRODUCT_ID']).mean()
df_growth_final = df_growth_final.reset_index()

df_growth_final.columns = [
    "PRODUCT_ID",
    "DATE_MONTH",
    "AVG_MONTHLY_GMV",
    "AVG_MONTHLY_ORDER_LINE",
    "AVG_MONTHLY_QTY_SOLD", 
    "AVG_MONTHLY_ORD_CUST",
    "GMV_GROWTH",
    "ORDER_LINE_GROWTH",
    "QTY_SOLD_GROWTH",
    "ORD_CUST_GROWTH"
]

For each `product_id`, mean value of the metric is computed to bring the metrics at product level

In [ ]:
df_growth_final.shape
df_growth_final.head().T

(91863, 10)

,0,1,2,3,4
PRODUCT_ID,3.859281e+06,3.859288e+06,3.859290e+06,3.859692e+06,3.860143e+06
DATE_MONTH,6.500000e+00,6.000000e+00,6.500000e+00,9.600000e+00,6.500000e+00
AVG_MONTHLY_GMV,7.603250e+01,1.106036e+02,9.066083e+01,1.536200e+01,3.542427e+03
AVG_MONTHLY_ORDER_LINE,1.433333e+01,1.563636e+01,3.216667e+01,1.800000e+00,9.395000e+02
AVG_MONTHLY_QTY_SOLD,1.508333e+01,1.600000e+01,3.641667e+01,1.800000e+00,1.066500e+03
AVG_MONTHLY_ORD_CUST,1.350000e+01,1.445455e+01,2.883333e+01,1.800000e+00,7.990833e+02
GMV_GROWTH,1.066986e-01,3.153868e-02,-5.790777e-02,8.361096e-02,6.902904e-03
ORDER_LINE_GROWTH,9.651241e-02,6.759113e-02,-1.056347e-01,8.333333e-02,6.407062e-03
QTY_SOLD_GROWTH,1.094146e-01,3.018294e-02,-6.561971e-02,8.333333e-02,4.861500e-03
ORD_CUST_GROWTH,6.450540e-02,1.017764e-01,-9.447413e-02,8.333333e-02,-2.362699e-03


In [ ]:
df_growth_final['PRODUCT_ID'].value_counts()

3859281     1
7738826     1
7738911     1
7738889     1
7738881     1
           ..
4019606     1
4019605     1
4019546     1
4019545     1
10759751    1
Name: PRODUCT_ID, Length: 91863, dtype: int64

### Save data - Interim

The data loaded using the query is transformed to include additional columns. The original data along with the new columns in saved at the `nbs/data/interim` folder.

In [ ]:
#hide
if use_snowflake:
    df_growth_metrics = pd.read_sql(con=con, sql = growth_metrics_query)
    df_growth_metrics.to_parquet('data/interim/growth_metrics.parquet')
else:
    df_growth_metrics = pd.read_parquet('data/interim/growth_metrics.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### First added to the cart

#### Query

In [ ]:
first_added_query = """
with cte as (
    select row_number() over (
        partition by user_id, session_id order by session_event_row_number
        ) as row_number,
        seg.event_date, seg.store_id, seg.metro_id, 
        seg.product_id, seg.order_number, p.name, ol.requested_qty,
ol.saved_product_price from
og_views.order_lines ol
inner join 
  (
  --keep the row that was added first for the same product product_id
  select 
    event_type, user_id, session_id,
    min(session_event_row_number) as session_event_row_number,
    event_date, platform, store_id,
    metro_id, cart_id, product_id, order_number from 
    (
    select car.*, som.order_number from 
      (
      --remove duplicate rows
      select event_type, event_type_secondary, event_id, user_id, session_id, 
      session_event_row_number, event_date, event_time, platform, store_id, 
      metro_id, cart_id, product_id
      from segment.segment_cart_add_remove
      group by event_type, event_type_secondary, event_id, user_id, session_id, 
      session_event_row_number, event_date, event_time, platform, store_id, 
      metro_id, cart_id, product_id
      having event_date between '2021-12-01' and '2021-12-31'
      and event_type = 'CART ADD ITEM'
      ) car
    inner join segment.segment_order_summary som
    on car.user_id = som.user_id and
    car.session_id = som.session_id
    )
  group by event_type, user_id, session_id, event_date, platform, store_id,
  metro_id, cart_id, product_id, order_number
  order by user_id, session_id, session_event_row_number
  ) seg
--inner join og_views.order_lines ol on
on ol.order_id = seg.order_number
and ol.requested_product_id= seg.product_id
left join og_views.products p
on p.id = ol.requested_product_id
where to_date(ol.created_at) between '2021-12-01' and '2021-12-31'
order by user_id, session_id, session_event_row_number)


SELECT  product_id, count(product_id)
FROM    cte
WHERE   row_number = 1
and     metro_id = 49
and     store_id = 10
group by 1
"""

#### Save data

In [ ]:
#hide
if use_snowflake:
    df_first_added = pd.read_sql(con=con, sql = first_added_query)
    df_first_added.to_parquet('data/raw/first_added.parquet')
else:
    df_first_added = pd.read_parquet('data/raw/first_added.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Convert `PRODUCT_ID` to integer type and set it as index to the `df_first_added`

In [ ]:
#hide
df_first_added['PRODUCT_ID'] = df_first_added['PRODUCT_ID'].astype(int)
df_first_added.set_index('PRODUCT_ID', inplace=True)
df_first_added.columns = ['FIRST_ADDED_COUNT']

In [ ]:
df_first_added.shape
df_first_added.head()

(18133, 1)

,FIRST_ADDED_COUNT
PRODUCT_ID,
3961305,8
3986831,20
3965215,46
3968330,3
3866509,2


There are 18,000 products that have been added at least once as the first product to the order

## KVI - key value indicators

### Query

In [ ]:
kvis_query = """
SELECT DISTINCT ol.actual_product_id AS PRODUCT_ID,
                p.NAME AS PRODUCT_NAME,
                sps.KVI
FROM PRD_DATALAKEHOUSE.OG_VIEWS.ORDER_LINES ol
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.ORDERS o ON ol.order_id = o.id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.METROS m ON m.id = o.metro_id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.STORES s ON s.id = o.store_id
INNER JOIN PRD_DATALAKEHOUSE.OG_VIEWS.PRODUCTS p ON p.id = ol.actual_product_id
LEFT JOIN PRD_DATALAKEHOUSE.OG_VIEWS.STORE_PRODUCT_SETTINGS sps ON sps.product_id = ol.actual_product_id
AND sps.store_id = o.store_id
AND sps.metro_id = o.metro_id
WHERE o.status ='delivered'
  AND ACTUAL_PRODUCT_TYPE = 'Product'
  AND is_external_platform_order = FALSE
  AND partner_id IS NULL
  AND convert_timezone('UTC', 'US/Central', DELIVERED_AT::timestamp_ntz)::date BETWEEN '2021-12-01' AND '2021-12-31'
  AND m.name = 'Detroit'
  AND s.name = 'Meijer';
"""

### Save data

In [ ]:
#hide
if use_snowflake:
    df_kvis = pd.read_sql(con=con, sql = kvis_query)
    df_kvis.to_parquet('data/raw/kvis.parquet')
else:
    df_kvis = pd.read_parquet('data/raw/kvis.parquet')

/Users/ravimulpuri/opt/miniconda3/envs/item_clustering/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Some of the KVI's available are

In [ ]:
df_kvis.shape
df_kvis.head()

(49788, 3)

,PRODUCT_ID,PRODUCT_NAME,KVI
0,3986421,Microwaveable Beef Stew,False
1,5772734,Alka-Seltzer Plus Maximum Strength Cold & Flu ...,False
2,3980192,"Sail Cooked Shrimp, Frozen",False
3,4047775,Conventional Oil 10W,False
4,4300863,Meijer Omega 3 Krill Oil,False


In [ ]:
df_kvis['KVI'].value_counts()
df_kvis['KVI'].mean()

False    48131
True      1604
Name: KVI, dtype: int64

0.0322509299286217

> Note: There are 1604 KVI's available which is 3.2% of the total number of products

## Combine data

Combine the average metrics data, click stream metrics data and growth metrics data at product level

In [ ]:
#hide
# Merge aavg_metrics with click_stream metric
df3 = df_avg_metrics.merge(df_clickstream_metrics, on=['PRODUCT_ID'], how='left', indicator=True)
df3.shape

(49991, 12)

In [ ]:
#hide
# drop products which dont have clickstream metrics
df3 = df3[df3['_merge'] == 'both']
final_df = df3.drop(columns=['PRODUCT_NAME_y', '_merge'])

In [ ]:
#hide
# Merge growth_metrics to the final_df
final_df = final_df.merge(df_growth_final, on=['PRODUCT_ID'], how='left', indicator=True)
final_df.shape

(47317, 20)

In [ ]:
#hide
# drop products which dont have growth metrics
# final_df has data for each product from order_financials, clickstream, and growth metrics
final_df = final_df.drop(columns=['_merge'])
final_df = final_df.dropna() 
final_df.set_index('PRODUCT_ID', inplace=True)

In [ ]:
#hide
# Merge df_first_added to the final_df and fill in missing values with 0
final_df = final_df.merge(df_first_added, on=['PRODUCT_ID'], how='left')
final_df['FIRST_ADDED_COUNT'] = final_df['FIRST_ADDED_COUNT'].fillna(0)

In [ ]:
final_df.shape
final_df.head()

(45206, 19)

,PRODUCT_NAME_x,AVG_GMV,AVG_ORDERLINES,AVG_QTY,AVG_COST,AVG_PRICE,AVG_CUST_COUNT,AVG_CART_ADD_DAILY,ADD_TO_ORDER_DAILY,DATE_MONTH,AVG_MONTHLY_GMV,AVG_MONTHLY_ORDER_LINE,AVG_MONTHLY_QTY_SOLD,AVG_MONTHLY_ORD_CUST,GMV_GROWTH,ORDER_LINE_GROWTH,QTY_SOLD_GROWTH,ORD_CUST_GROWTH,FIRST_ADDED_COUNT
PRODUCT_ID,,,,,,,,,,,,,,,,,,,
10433693,""" Handcrafted Ornament Set, 1"""" RoundDecember ...",7.590000,1.000000,1.000000,6.990000,7.590000,1.000000,1.000000,0.000000,11.500000,7.590000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0
4264419,"""I and love and you"" Cat Can Variety Pack",66.360000,1.000000,4.000000,13.990000,16.590000,1.000000,1.666667,0.333333,7.166667,73.538333,2.666667,4.500000,2.500000,1.464564,0.600000,1.453333,0.420000,0.0
8381569,"""JIFFY"" Honey Corn Muffin Mix",3.926471,2.411765,4.411765,1.664118,2.146471,2.352941,3.476190,0.190476,6.500000,39.597500,23.416667,41.916667,22.166667,0.168529,0.167238,0.202161,0.162516,0.0
9820133,"""This RITZ Sauté Black Honeycomb Solid Dual Ki...",3.790000,1.000000,1.000000,3.490000,3.790000,1.000000,1.000000,0.000000,10.000000,7.285000,1.000000,1.500000,1.000000,-0.648423,0.000000,-0.500000,0.000000,0.0
3945853,#10 Security Boxed Envelopes,2.934545,1.090909,1.090909,2.716364,2.934545,1.090909,1.166667,0.083333,6.500000,21.520000,7.750000,8.000000,7.750000,0.479654,0.458442,0.479654,0.458442,0.0


## Numeric columns

`PRODUCT_NAME` cant be converted to numeric type. It is a categorical column. So it is not included in the scaling process

In [ ]:
numerical_cols = [c for c in final_df.columns if final_df[c].dtype in ['int64', 'float64']]
numerical_cols

['AVG_GMV',
 'AVG_ORDERLINES',
 'AVG_QTY',
 'AVG_COST',
 'AVG_PRICE',
 'AVG_CUST_COUNT',
 'AVG_CART_ADD_DAILY',
 'ADD_TO_ORDER_DAILY',
 'DATE_MONTH',
 'AVG_MONTHLY_GMV',
 'AVG_MONTHLY_ORDER_LINE',
 'AVG_MONTHLY_QTY_SOLD',
 'AVG_MONTHLY_ORD_CUST',
 'GMV_GROWTH',
 'ORDER_LINE_GROWTH',
 'QTY_SOLD_GROWTH',
 'ORD_CUST_GROWTH',
 'FIRST_ADDED_COUNT']

## Scaling

Scaling the data to bring the data to the same scale for all the columns

### Standard scaling

Standard scaling uses the mean and standard deviation to scale the data. `DATE_MONTH` is also normalized. The variable varies linearly for each product. So this may not be a good way to scale this column

- Data before scaling

In [ ]:
final_df.shape
final_df[numerical_cols].head().T

(45206, 19)

PRODUCT_ID,10433693,4264419,8381569,9820133,3945853
AVG_GMV,7.59,66.360000,3.926471,3.790000,2.934545
AVG_ORDERLINES,1.00,1.000000,2.411765,1.000000,1.090909
AVG_QTY,1.00,4.000000,4.411765,1.000000,1.090909
AVG_COST,6.99,13.990000,1.664118,3.490000,2.716364
AVG_PRICE,7.59,16.590000,2.146471,3.790000,2.934545
AVG_CUST_COUNT,1.00,1.000000,2.352941,1.000000,1.090909
AVG_CART_ADD_DAILY,1.00,1.666667,3.476190,1.000000,1.166667
ADD_TO_ORDER_DAILY,0.00,0.333333,0.190476,0.000000,0.083333
DATE_MONTH,11.50,7.166667,6.500000,10.000000,6.500000
AVG_MONTHLY_GMV,7.59,73.538333,39.597500,7.285000,21.520000


In [ ]:
#hide
from scipy import stats
# the first column is product_name. So we drop it
final_df_std_scaling = stats.zscore(final_df[numerical_cols])

- Data after scaling

In [ ]:
final_df_std_scaling.shape
final_df_std_scaling.head().T

(45206, 18)

PRODUCT_ID,10433693,4264419,8381569,9820133,3945853
AVG_GMV,-0.268471,1.820673,-0.398701,-0.403553,-0.433962
AVG_ORDERLINES,-0.160189,-0.160189,0.048793,-0.160189,-0.146732
AVG_QTY,-0.092161,0.059573,0.080400,-0.092161,-0.087563
AVG_COST,-0.212606,0.153996,-0.491531,-0.395907,-0.436423
AVG_PRICE,-0.241144,0.177262,-0.494210,-0.417804,-0.457573
AVG_CUST_COUNT,-0.158776,-0.158776,0.042060,-0.158776,-0.145281
AVG_CART_ADD_DAILY,-0.102077,-0.064090,0.039020,-0.102077,-0.092580
ADD_TO_ORDER_DAILY,-0.298786,0.522763,0.170670,-0.298786,-0.093399
DATE_MONTH,3.338651,0.080649,-0.420582,2.210881,-0.420582
AVG_MONTHLY_GMV,-0.245118,-0.163668,-0.205587,-0.245495,-0.227914


### Robust scaling

Robust scaling uses the median and IQR to scale the data. The scaling is more robust to outliers. 

In [ ]:
#hide
from sklearn.preprocessing import RobustScaler

transformer = RobustScaler(quantile_range=(2, 98)).fit((final_df[numerical_cols]))
final_df_robust_scaling = transformer.transform(final_df[numerical_cols])
final_df_robust_scaling = pd.DataFrame(final_df_robust_scaling, columns=final_df[numerical_cols].columns)

After scaling the data using robust scaling

In [ ]:
final_df_robust_scaling.shape
final_df_robust_scaling.head().T

(45206, 18)

,0,1,2,3,4
AVG_GMV,-0.028948,0.895536,-0.086578,-0.088724,-0.102181
AVG_ORDERLINES,-0.016240,-0.016240,0.144249,-0.016240,-0.005905
AVG_QTY,-0.033512,0.207775,0.240893,-0.033512,-0.026200
AVG_COST,0.000861,0.144362,-0.108321,-0.070890,-0.086749
AVG_PRICE,-0.008932,0.151849,-0.106179,-0.076818,-0.092100
AVG_CUST_COUNT,-0.014313,-0.014313,0.140604,-0.014313,-0.003904
AVG_CART_ADD_DAILY,-0.026436,0.022030,0.153578,-0.026436,-0.014319
ADD_TO_ORDER_DAILY,0.000000,0.333333,0.190476,0.000000,0.083333
DATE_MONTH,0.862069,0.114943,0.000000,0.603448,0.000000
AVG_MONTHLY_GMV,-0.031231,0.013180,-0.009676,-0.031436,-0.021850


## Save data

Save the data to nbs/data folder

In [ ]:
#hide
if use_snowflake:
    # final_dfs
    final_df.to_parquet('data/processed/final_df.parquet')
    final_df_std_scaling.to_parquet('data/processed/final_df_std_scaling.parquet')
    final_df_std_scaling.to_parquet('data/processed/final_df_robust_scaling.parquet')
    

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_data.ipynb.
Converted 01_analysis.ipynb.
Converted 02_models.ipynb.
Converted 03_feature_importance.ipynb.
Converted 04_validation.ipynb.
Converted index.ipynb.
